In [4]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep

sagemaker.__version__

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


'2.215.0'

# create mlops deployment project 

In [5]:
sc = boto3.client("servicecatalog")
sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model deployment"

In [6]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [7]:
p_ids

['prod-246h53r45lvzo']

In [8]:
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

ML Ops product id: prod-246h53r45lvzo


In [9]:
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [10]:
provisioning_artifact_id

'pa-hufmcjsaiqzoq'

In [17]:
# project_name = f"bits-deploy-{strftime('%-m-%d-%H-%M-%S', gmtime())}"
project_name = f"bits-deploy-new"


In [18]:
model_package_group_name= "bits-webminar-june-24-xgboost-model-group"


In [19]:
project_parameters = [
    {
        'Key': 'SourceModelPackageGroupName',
        'Value': model_package_group_name
    },
]

In [20]:
sm = boto3.client("sagemaker")
sc = boto3.client("servicecatalog")

In [21]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:eu-north-1:058264393695:project/bits-deploy-new', 'ProjectId': 'p-4bzgqm9ro6ew', 'ResponseMetadata': {'RequestId': 'a73ef337-122c-4e4a-a4ea-5a82d13fa277', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a73ef337-122c-4e4a-a4ea-5a82d13fa277', 'content-type': 'application/x-amz-json-1.1', 'content-length': '111', 'date': 'Thu, 27 Jun 2024 15:15:58 GMT'}, 'RetryAttempts': 0}}


In [22]:
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    sleep(10)
    print("Waiting for project creation completion")

print(f"MLOps project {project_name} creation completed")

Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
Waiting for project creation completion
MLOps project bits-deploy-new creation completed


In [21]:
# list all model packages and select the latest one
model_packages = []

for p in sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])

if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group")
    
latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

arn:aws:sagemaker:eu-north-1:058264393695:model-package/bits-webminar-june-24-xgboost-model-group/2


In [22]:
# approve the latest model 
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

In [23]:
print(project_name)
print(project_id)

bits-deploy
p-frtceok62sg7


In [24]:
# Get project id
project_id = sm.describe_project(ProjectName=project_name)['ProjectId']

# Construct the CodePipline pipeline name
code_pipeline_name = f"sagemaker-{project_name}-{project_id}-modeldeploy"

In [23]:
region = "us-east-1"

In [24]:
from IPython.core.display import display, HTML

# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/{}/view?region={}">AWS CodePipeline</a></b>'.format(
            code_pipeline_name, region)
    )
)

/tmp/ipykernel_19/1467144251.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
